In [1]:
# Test on German Credit Dataset using the QASM backend simulator
import numpy as np
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

provider = IBMQ.load_account()
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
import time
seed = 12345
algorithm_globals.random_seed = seed
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import  PCA
from sklearn.model_selection import train_test_split

from qiskit.providers.ibmq import least_busy


# In[3]:
# prepare to send standard output to a text file
import sys

orig_stdout = sys.stdout
fl1 = open('out_pca16qasm.csv', 'w')
sys.stdout = fl1


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#balanced data preparation
credit=pd.read_csv('German_Preprocessed.csv').sample(frac=1,random_state=3)
credit = credit.groupby('classification').head(300).sample(frac=1,random_state=3)
X_or, y_or = credit.drop('classification',axis=1).values, credit.classification.squeeze().values
y_or=[i if i ==1 else 0 for i in y_or]
X_train=X_or[:500,:]
X_test=X_or[500:,:]
Y_train=y_or[:500]
Y_test=y_or[500:]
scaler = MinMaxScaler(feature_range=(-1,1))
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# In[4]:

def get_acc(dim,reps,cost,classic=False,verbose=True,returnAll=False,   maps='ZZ',ent='full',random_state=3,simulator='statevector_simulator',is_final='False',        use_pca=True):
    time0=time.time()
    scaler = MinMaxScaler(feature_range=(-1,1))
    X=X_train.copy()
    y=Y_train.copy()
    if maps=='ZZ':
        adhoc_feature_map = ZZFeatureMap(feature_dimension=dim, 
                                         reps=reps, entanglement=ent)
    else:
        adhoc_feature_map = ZFeatureMap(feature_dimension=dim, 
                                         reps=reps)
    #get backend: qasm or statevector. shots useless for statevector
    adhoc_backend = QuantumInstance(BasicAer.get_backend(simulator), shots=512,
                                    seed_simulator=seed, seed_transpiler=seed)
    adhoc_kernel = QuantumKernel(feature_map=adhoc_feature_map, quantum_instance=adhoc_backend)

    #usiamo solo n/1000 osservazioni
    if is_final==True:
        train_x, test_x, train_y, test_y= X,X_test.copy(),y,Y_test.copy()
    else:
        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=random_state)
    if use_pca==True:
        pca = PCA(dim)
        train_x=pca.fit_transform(train_x)
        test_x=pca.transform(test_x)
        scaler2=MinMaxScaler(feature_range=(-1,1))
        train_x=scaler2.fit_transform(train_x)
        test_x=scaler2.transform(test_x)
    print(pca.explained_variance_)

    if is_final==True:
        print(f'Train x {train_x.shape} and train y {len(train_y)}')
        print(f'Test x {test_x.shape} and test y {len(test_y)}')
    adhoc_svc = SVC(C=cost,kernel=adhoc_kernel.evaluate)
    if classic==True:
        adhoc_svc = SVC(C=cost,kernel='rbf')
    adhoc_svc.fit(train_x, train_y)
    adhoc_score = adhoc_svc.score(test_x, test_y)
    adhoc_score_train = adhoc_svc.score(train_x, train_y)
    time1=time.time()
    if verbose:
        print(f'Total time:{"{:.2}".format(time1-time0)}\n        Callable kernel classification test score: {adhoc_score}\n        Callable kernel classification train score: {adhoc_score_train}\n        Parameters: PCA {dim}, reps {reps},cost {cost}\n        total of 1s: {sum(y)/len(y)}\n\n')
    if returnAll:
        return train_x,train_y,test_x, test_y, adhoc_svc,adhoc_score, adhoc_score_train
    else:
        return adhoc_score, adhoc_score_train



In [3]:
# In[5]:


acc_classica=[]
acc_quant_zz_full_1=[]
acc_quant_zz_full_2=[]
acc_quant_zz_linear_1=[]
acc_quant_zz_linear_2=[]
acc_quant_z_1=[]
acc_quant_z_2=[]
pca=[2,3,4,6,8,10,12,16]
pca=[16]
list_cost=np.logspace(-1,1,5)
# pca=[2,3,4,6,8,10,12]
# list_cost=np.logspace(2,5,20)



In [4]:
pca


[16]

In [ ]:
#accuracy zfeaturemap 1 rep qasm
acc_quant_z_1_qasm=[]
for i in pca:
    lista_tmp=[]
    for c in list_cost:
        lista_cv=[]
        for y in [3]:
            lista_cv.append(get_acc(i,1,c,classic=False,verbose=True,simulator='qasm_simulator',maps='Z',random_state=y)[0])
        lista_tmp.append(np.mean(lista_cv))
    cost_value=list_cost[np.argmax(lista_tmp)]
    acc_quant_z_1_qasm.append(get_acc(i,1,cost_value,classic=False,verbose=True,simulator='qasm_simulator',maps='Z',        random_state=y,is_final=True)[0])
# reset the original stdout and close the output file
sys.stdout = orig_stdout
f.close()